In [1]:
import os, sys
# 
# os.environ['KMP_DUPLICATE_LIB_OK']='True' # uncomment this line if omp error occurs on OSX for python 3
# os.environ["OMP_NUM_THREADS"] = "4" # export OMP_NUM_THREADS=4
# os.environ["OPENBLAS_NUM_THREADS"] = "4" # export OPENBLAS_NUM_THREADS=4 
# os.environ["MKL_NUM_THREADS"] = "6" # export MKL_NUM_THREADS=6
# os.environ["VECLIB_MAXIMUM_THREADS"] = "4" # export VECLIB_MAXIMUM_THREADS=4
# os.environ["NUMEXPR_NUM_THREADS"] = "6" # export NUMEXPR_NUM_THREADS=6
import numpy as np
print(np.__version__)
from scipy import linalg
import stomp_functions as stf
from quspin.operators import hamiltonian, commutator
from quspin.basis import spinful_fermion_basis_general
import matplotlib.pyplot as plt
import matplotlib as mpl
from qiskit.quantum_info import random_clifford



1.26.4
1.26.4


In [2]:
# Set font size of plot elements
SMALL_SIZE = 12
MEDIUM_SIZE = 14
BIGGER_SIZE = 18

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [3]:
# define model parameters
Lx, Ly = 2, 2 # linear dimension of spin 1 2d lattice
N_2d = Lx * Ly # number of sites for spin 1

J = 1.0 # hopping matrix element
U = 2.0 # onsite interaction
mu = 0.5 # chemical potential

In [4]:
# Create symmetry transformations for 2D lattice
s = np.arange(N_2d)
x = s % Lx    # x position of sites
y = s // Lx   # y position of sites

Tx = (x + 1) % Lx + Lx * y   # translation along x direction
Ty = x + Lx * ((y + 1) % Ly) # translation along y-direction
Px = x + Lx * ( Ly - y - 1) # reflection about x-axis
Py = (Lx - x - 1) + Lx * y  # reflection about y-axis
S = -(s + 1) # fermion spin inversion in the simple case

In [5]:
# Define number of up and down fermions in lattice
# We want to be a bit above half-filling here
N_up = N_2d // 2 + 0
N_down = N_2d // 2

# Create basis
basis_2d=spinful_fermion_basis_general(N_2d, Nf=(N_up, N_down), double_occupancy=True,
                                       #kxblock=(Tx,0), kyblock=(Ty,0),
                                       #pxblock=(Px,1), pyblock=(Py,0), # contains GS
                                       #sblock=(S,0)
                                      )
print(basis_2d)

reference states: 
array index   /   Fock state   /   integer repr. 
  0.  |1 1 0 0>|1 1 0 0>  204 
  1.  |1 1 0 0>|1 0 1 0>  202 
  2.  |1 1 0 0>|1 0 0 1>  201 
  3.  |1 1 0 0>|0 1 1 0>  198 
  4.  |1 1 0 0>|0 1 0 1>  197 
  5.  |1 1 0 0>|0 0 1 1>  195 
  6.  |1 0 1 0>|1 1 0 0>  172 
  7.  |1 0 1 0>|1 0 1 0>  170 
  8.  |1 0 1 0>|1 0 0 1>  169 
  9.  |1 0 1 0>|0 1 1 0>  166 
 10.  |1 0 1 0>|0 1 0 1>  165 
 11.  |1 0 1 0>|0 0 1 1>  163 
 12.  |1 0 0 1>|1 1 0 0>  156 
 13.  |1 0 0 1>|1 0 1 0>  154 
 14.  |1 0 0 1>|1 0 0 1>  153 
 15.  |1 0 0 1>|0 1 1 0>  150 
 16.  |1 0 0 1>|0 1 0 1>  149 
 17.  |1 0 0 1>|0 0 1 1>  147 
 18.  |0 1 1 0>|1 1 0 0>  108 
 19.  |0 1 1 0>|1 0 1 0>  106 
 20.  |0 1 1 0>|1 0 0 1>  105 
 21.  |0 1 1 0>|0 1 1 0>  102 
 22.  |0 1 1 0>|0 1 0 1>  101 
 23.  |0 1 1 0>|0 0 1 1>   99 
 24.  |0 1 0 1>|1 1 0 0>   92 
 25.  |0 1 0 1>|1 0 1 0>   90 
 26.  |0 1 0 1>|1 0 0 1>   89 
 27.  |0 1 0 1>|0 1 1 0>   86 
 28.  |0 1 0 1>|0 1 0 1>   85 
 29.  |0 1 0 1>|0 0 1 1>   83 
 

In [6]:
# Set up hamiltonian
hopping_left = [[-J, i, Tx[i]] for i in range(N_2d)] + [[-J, i, Ty[i]] for i in range(N_2d)]
hopping_right = [[+J, i, Tx[i]] for i in range(N_2d)] + [[+J, i, Ty[i]] for i in range(N_2d)]
potential = [[-mu, i] for i in range(N_2d)]
interaction = [[U, i, i] for i in range(N_2d)]

static=[["+-|",hopping_left], # spin up hops to left
		["-+|",hopping_right], # spin up hops to right
		["|+-",hopping_left], # spin down hopes to left
		["|-+",hopping_right], # spin up hops to right
		["n|",potential], # onsite potenial, spin up
		["|n",potential], # onsite potential, spin down
		["n|n",interaction]] # spin up-spin down interaction

In [7]:
# Build the Hamiltonian
H = hamiltonian(static, [], basis=basis_2d, dtype=np.float64).toarray()

Hermiticity check passed!
Symmetry checks passed!


C:\Users\gerar\AppData\Local\Temp\ipykernel_27780\1336399668.py:2: UserWarning: Test for particle conservation not implemented for <class 'quspin.basis.basis_general.fermion.spinful_fermion_basis_general'>, to turn off this warning set check_pcon=False in hamiltonian
  H = hamiltonian(static, [], basis=basis_2d, dtype=np.float64).toarray()


In [8]:
E = linalg.eigvalsh(H)

In [9]:
# Create initial state
np.random.seed(5)
init_state = np.random.random(H.shape[0]) + 1j * np.random.random(H.shape[0])
init_state /= np.sqrt(np.sum(np.abs(init_state) ** 2))
#init_state = np.zeros(H.shape[0])
#init_state[0] = 1

beta = 0.3
num_steps = 500
betas, dt = np.linspace(0, beta, num_steps, retstep=True)

In [10]:
ovlps, H_ovlps = stf.classical_calc(init_state, H, H, num_steps, dt)

In [11]:
stomp_results = {}
#for i, _ in enumerate(keys):
#    stomp_results[_] = stomp_energies[i]
#stomp_results['E_0'] = stomp_energies[0]   
stomp_results['Exact'] = E
stomp_results['Z'] = ovlps
stomp_results['O'] = H_ovlps

In [12]:
main_path = "2D_FH_clifford_testing/"
filename = "2DFH_N2d=" + str(N_2d) + "_Nferm=" + str(N_up + N_down) \
            +"_Numsteps=" + str(num_steps) + "_t=" + str(beta) +".npz"
np.savez(main_path + filename, **stomp_results)